In [1]:
import torch
import torch.nn as nn
from sinabs.from_torch import from_model
from sinabs.backend.dynapcnn import DynapcnnNetworkGraph
from sinabs.layers import Merge, IAFSqueeze

In [2]:
torch.manual_seed(0)

In [3]:
channels = 1
height = 28
width = 28

input_shape = (channels, height, width)

## Network Module (Model #1)

This one won't pass the `config_builder.validate_configuration` because the nodes inputing into the `Merge` layer don't have the same output dimension.

In [4]:
class SNN(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.conv1 = nn.Conv2d(1, 10, 5, 1, bias=False)
        self.iaf1 = IAFSqueeze(batch_size=1)
        self.pool1 = nn.AvgPool2d(2,2)

        self.conv2 = nn.Conv2d(10, 20, 5, 1, bias=False)
        self.iaf2 = IAFSqueeze(batch_size=1)
        self.pool2 = nn.AvgPool2d(2,2)

        self.conv3 = nn.Conv2d(20, 1, 3, 1, bias=False)
        self.iaf3 = IAFSqueeze(batch_size=1)
        self.pool3 = nn.AvgPool2d(2,2)

        self.flat = nn.Flatten()

        self.fc1 = nn.Linear(121, 500, bias=False)
        self.iaf4 = IAFSqueeze(batch_size=1)
        self.fc2 = nn.Linear(500, 10, bias=False)
        self.iaf5 = IAFSqueeze(batch_size=1)

        self.adder = Merge()

    def forward(self, x):
        
        con1_out = self.conv1(x)
        iaf1_out = self.iaf1(con1_out)
        pool1_out = self.pool1(iaf1_out)

        conv2_out = self.conv2(pool1_out)
        iaf2_out = self.iaf2(conv2_out)
        pool2_out = self.pool2(iaf2_out)

        conv3_out = self.conv3(self.adder(iaf1_out, pool2_out))
        iaf3_out = self.iaf3(conv3_out)
        pool3_out = self.pool3(iaf3_out)

        flat_out = self.flat(pool3_out)
        
        fc1_out = self.fc1(flat_out)
        iaf4_out = self.iaf4(fc1_out)
        fc2_out = self.fc2(iaf4_out)
        iaf5_out = self.iaf5(fc2_out)

        return iaf5_out

In [5]:
snn = SNN()

In [6]:
x = torch.randn((1, *input_shape))

con1_out = snn.conv1(x)
iaf1_out = snn.iaf1(con1_out)
print('iaf1_out: ', iaf1_out.shape)
pool1_out = snn.pool1(iaf1_out)

conv2_out = snn.conv2(pool1_out)
iaf2_out = snn.iaf2(conv2_out)
pool2_out = snn.pool2(iaf2_out)
print('pool2_out: ', pool2_out.shape)

added = snn.adder(iaf1_out, pool2_out)
print('added: ', added.shape)

conv3_out = snn.conv3(added)
iaf3_out = snn.iaf3(conv3_out)
pool3_out = snn.pool3(iaf3_out)
print('pool3_out: ', pool3_out.shape)

flat_out = snn.flat(pool3_out)
print('flat_out: ', flat_out.shape)

fc1_out = snn.fc1(flat_out)
iaf4_out = snn.iaf4(fc1_out)
fc2_out = snn.fc2(iaf4_out)
iaf5_out = snn.iaf5(fc2_out)

In [7]:
print(f'DynapcnnLayer 0 [core 0]:')
print(f'                                                input: {x.shape}')
con1_out = snn.conv1(x)
print(f'                                                conv1: {con1_out.shape}')
iaf1_out = snn.iaf1(con1_out)
print(f'                                                iaf1: {iaf1_out.shape}')
pool1_out = snn.pool1(iaf1_out)
print(f'                                                pool1: {pool1_out.shape}\n')

print(f'DynapcnnLayer 1 [core 1]:')
print(f'                                                input: {pool1_out.shape}')
conv2_out = snn.conv2(pool1_out)
print(f'                                                conv2: {conv2_out.shape}')
iaf2_out = snn.iaf2(conv2_out)
print(f'                                                iaf2: {iaf2_out.shape}')
pool2_out = snn.pool2(iaf2_out)
print(f'                                                pool2: {pool2_out.shape}\n')

added = snn.adder(iaf1_out, pool2_out)

print(f'DynapcnnLayer 2 [core 2]:')
print(f'                                                input: {added.shape}')
conv3_out = snn.conv3(added)
print(f'                                                conv3: {conv3_out.shape}')
iaf3_out = snn.iaf3(conv3_out)
print(f'                                                iaf3: {iaf3_out.shape}')
pool3_out = snn.pool3(iaf3_out)
print(f'                                                pool3: {pool3_out.shape}')

flat_out = snn.flat(pool3_out)

fc1_out = snn.fc1(flat_out)
iaf4_out = snn.iaf4(fc1_out)
print(f'DynapcnnLayer 3: {iaf4_out.shape}')

fc2_out = snn.fc2(iaf4_out)
iaf5_out = snn.iaf5(fc2_out)
print(f'DynapcnnLayer 4: {iaf5_out.shape}')

## DynapCNN Model

In [8]:
hw_model = DynapcnnNetworkGraph(
    snn,
    discretize=True,
    input_shape=input_shape
)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x625 and 16x500)

In [ ]:
hw_model.to(device="speck2edevkit:0")

Layer[2] input size x: 24 different than Layer[0] output size x: 24 pooling: 2
Layer[2] input size y: 24 different than Layer[0] output size y: 24 pooling: 2
Layer[2] input size x: 24 different than Layer[1] output size x: 8 pooling: 2
Layer[2] input size y: 24 different than Layer[1] output size y: 8 pooling: 2



ValueError: Generated config is not valid for speck2edevkit:0